In [8]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import sqlite3

In [2]:
headers = {'User-Agent': 'Mozilla/5.0'}
urls = ["https://www.reddit.com/r/DebateAVegan/comments/6xtq3v/fake_meat_products/"]

# ,
#        "https://www.reddit.com/r/vegan/comments/7s81qn/how_do_you_feel_about_fake_meat/",
#        "https://www.reddit.com/r/vegetarian/comments/1fpvwv/best_fake_meat/",
#        "https://www.reddit.com/r/vegetarian/comments/5oqulb/what_are_your_favorite_faux_meat_products_and_why/",
#        "https://www.reddit.com/r/vegetarian/comments/a1u03x/any_other_vegetarians_dislike_fake_meat_products/",
#        "https://www.reddit.com/r/unpopularopinion/comments/kdmtmx/fake_meat_is_dumb/"

In [3]:
def scrape_reddit(url, headers):
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    comments = soup.find_all('p')
    clean_comments = [comment.get_text() for comment in comments if len(comment.get_text().split()) > 2]
    
    title = soup.find("h1").get_text()
    
    return (title, clean_comments)
    

In [9]:
def write_to_db(title, clean_comments):
    conn = sqlite3.connect("reviewdb.db")
    c = conn.cursor()
    c.execute("DROP TABLE IF EXISTS reddit_thread")
    c.execute("CREATE TABLE reddit_thread ('title', 'comment')")
    
    for clean_comment in clean_comments:
        c.execute("INSERT INTO reddit_thread ('title', 'comment') VALUES (?, ?)", 
                  (title, clean_comment))
        
    conn.commit()
    conn.close()

In [10]:
for url in urls:
    (title, clean_comments) = scrape_reddit(url, headers)
    print(title)
    print(clean_comments)
    write_to_db(title, clean_comments)

Fake "Meat" products.
['I have recently switched to a plant based diet but will never consider myself a vegan ... I still enjoy fly fishing and wear leather boots.', 'Guess my question is if eating meat is so horrible why is so much effort spent on replacing meat with other "things" that are suppose to taste like meat, look like meat. Which I have read countless times looks like rotting corpse.', "Just confuses me why... I understand it's the lesser of two evils in (your) mind but why make it look like meat at all or name it after the animals it's suppose to taste like.", 'Thank you for your submission! Some topics come up a lot in this subreddit, so we would like to remind everyone to use the search function and to check out the wiki before creating a new post.', "When participating in a discussion, try to be as charitable as possible when replying to arguments. If an argument sounds ridiculous to you, consider that you may have misinterpreted what the author was trying to say. Ask cl

In [11]:
conn = sqlite3.connect("reviewdb.db")
c = conn.cursor()
c.execute("SELECT * FROM reddit_thread")
output = c.fetchall()
conn.close()
print(output)

[('Fake "Meat" products.', 'I have recently switched to a plant based diet but will never consider myself a vegan ... I still enjoy fly fishing and wear leather boots.'), ('Fake "Meat" products.', 'Guess my question is if eating meat is so horrible why is so much effort spent on replacing meat with other "things" that are suppose to taste like meat, look like meat. Which I have read countless times looks like rotting corpse.'), ('Fake "Meat" products.', "Just confuses me why... I understand it's the lesser of two evils in (your) mind but why make it look like meat at all or name it after the animals it's suppose to taste like."), ('Fake "Meat" products.', 'Thank you for your submission! Some topics come up a lot in this subreddit, so we would like to remind everyone to use the search function and to check out the wiki before creating a new post.'), ('Fake "Meat" products.', "When participating in a discussion, try to be as charitable as possible when replying to arguments. If an argume